## Imports

In [90]:
# Pandas, numpy, matplotlib and seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# RiotWatcher
from riotwatcher import LolWatcher, ApiError

# OS tools
import os
import shutil
import json
import sys

# Custom scripts
from extract_players_performance import extract_players_performance
from remove_perks import remove_perks
from cleaner import replace_champ_names_with_tags
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

## Global Variables

In [3]:
# Global Variables

# Key to gain access to Riot API's data
api_key = 'RGAPI-458748cb-0829-47d6-a3dc-517b6f898a38'

# Instance of the wrapper API 'RiotWatcher'
watcher = LolWatcher(api_key)

# Region of the player
region = 'na1'

# Name of the player's account
account_name = 'Jedı07'

# PUUID linked to the player's account
puuid = None

## Get an Account

In [6]:
# Get account
account = watcher.summoner.by_name(region, account_name)

# Save puuid
puuid = account['puuid']

## Get Match History

In [7]:
# Get match history data
# Note: max count=100
match_history = watcher.match.matchlist_by_puuid(region, puuid, queue=420, count=100)

print(match_history)

['NA1_4276627830', 'NA1_4276630734', 'NA1_4271399837', 'NA1_4271341235', 'NA1_4271285171', 'NA1_4200069215', 'NA1_4200057973', 'NA1_4200023211', 'NA1_4195249549', 'NA1_4195296421', 'NA1_4192896758', 'NA1_4192828915', 'NA1_4192861697', 'NA1_4192737361', 'NA1_4191628052', 'NA1_4190571787', 'NA1_4190503402', 'NA1_4190491622', 'NA1_4183313170', 'NA1_4183208690', 'NA1_4183220508', 'NA1_4181962985', 'NA1_4181858654', 'NA1_4178015004', 'NA1_4177827642', 'NA1_4176610684', 'NA1_4176591844', 'NA1_4176486747', 'NA1_4175591432', 'NA1_4175571307', 'NA1_4174230478', 'NA1_4174137022', 'NA1_4172646223', 'NA1_4172587470', 'NA1_4171197980', 'NA1_4171203080', 'NA1_4169900567', 'NA1_4169821877', 'NA1_4169772592', 'NA1_4102698720', 'NA1_4102683102', 'NA1_4102635137', 'NA1_4100010194', 'NA1_4099965698', 'NA1_4099991612', 'NA1_4099917364', 'NA1_4099933969', 'NA1_4091819791', 'NA1_4091752285', 'NA1_4090225331', 'NA1_4090232653', 'NA1_4048993419', 'NA1_4048908141', 'NA1_4048932745', 'NA1_4048887179', 'NA1_4048

## Get Data of a Specific Match

In [12]:
# ID of the match as per the previous cell's match ID list
match_id = match_history[0]

# Get the match's data
match_data = watcher.match.by_id(region, match_id)

## Get Timeline of the Match

In [13]:
# Get timeline of the match specific in the previous cell
match_timeline = watcher.match.timeline_by_match(region, match_id)

## Create Directories to Store Match History

In [14]:
# Directory of this player's match data
general_dir = '../data/' + account_name + '_match_history'

# Create general directory
if os.path.exists(general_dir):
    shutil.rmtree(general_dir)
os.mkdir(general_dir)

# Save match history as json
match_history_file_path = general_dir + '/' + account_name + '_match_history.json'
match_history_json_string = json.dumps(match_history, indent=2)
match_history_json_file = open(match_history_file_path, 'w')
match_history_json_file.write(match_history_json_string)
match_history_json_file.close()

## Create Directories and Store Individual Match Data

In [15]:
# Directory of this player's match data
matches_dir = general_dir + '/matches'

# Create matches directory
if os.path.exists(matches_dir):
    shutil.rmtree(matches_dir)
os.mkdir(matches_dir)

# Create directories for every match in the match history
directory_name = 'match_'

for match in match_history:

    # Create directory for specific match
    new_directory_name = directory_name + match
    
    match_dir = matches_dir + '/' + new_directory_name

    if os.path.exists(match_dir):
        shutil.rmtree(match_dir)
    os.mkdir(match_dir)

    # Get match data
    match_data = watcher.match.by_id(region, match)
    match_file_path = match_dir + '/' + match + '.json'

    # Save match data
    match_data_json_string = json.dumps(match_data, indent=2)
    match_data_json_file = open(match_file_path, 'w')
    match_data_json_file.write(match_data_json_string)
    match_data_json_file.close()  

## Execute Script to Separate Individual Player Data

In [16]:
extract_players_performance(account_name)

## Execute Script to Remove Perks (Runes) from Player Data

In [17]:
remove_perks(account_name)

## Get More Match Data from Players Encountered

In [20]:
# Start by getting the puuid of every unique player encountered

match_dir = '../data/Jedı07_match_history/matches/match_'

match_participants = []

for match in match_history:
    match_path = match_dir + match + '/participants'

    with os.scandir(match_path) as iter:
        for directory in iter:
            if not (directory.name.startswith('.') and directory.is_file()):
                if directory.name not in match_participants:
                    match_participants.append(directory.name)

for index in range(len(match_participants)-1):
    match_participants[index] = match_participants[index][7:]

print(len(match_participants))

797


In [27]:
# Get the account names of every player encountered

players_encountered = []


for index in range(len(match_participants)-1):
    players_encountered.append(watcher.summoner.by_puuid(region, match_participants[index])['name'])

#print(players_encountered)


['Plus3Hundo', 'Homiechillax', 'Fencepuppet', 'Cermit365', 'blikblak', 'Will BB', 'àntx', 'Diu Da Will BB', 'Domz01', 'Jedı07', 'SugarHighs', 'Tuna Davis', 'Mattyboyywonder', 'chimuru', '24oz Cold Brew', 'Jessefcd', 'SunxRise', 'YTyukii', 'Nov4yn', 'jetstream jordan', 'Inconvenience', 'Unknown Father', 'Znulodsa', 'Rydur01', 'Mega Adhesive', 'hehexd tyler1', 'chubzhuh', 'ForeverDark2012', 'Issvc', 'Bryanxq3', 'cantivskeitaro', 'Hardsruck', 'hiwi', 'Ozymandias774', 'PostViral', 'The Cosby Suite', 'Zatagon', 'cutiejam', 'PrettyCuteDaddy', 'TheLordEsus', 'zhlhahaha', 'DrDoughnutDude', 'Natebgreat', 'maorT', 'Phangphxn', 'Šampp', 'Jiimoto', 'o Wayfinder o', 'Daddy Skii', 'Skeptical Otter', 'drkeagle98', 'Thaifi', 'Outcome', 'Kraken Mutes ALL', 'Mr Chiwi', 'LCS Miracle Run', 'Harpooneerx', 'Wabby Babbo', 'FrostyCake', 'DooDoo Deluxe', 'ZhouXingwen', 'ShinigamiGhoul', 'BumsAreCool', 'THEGTCGU111', 'Bobgrey', 'SlenderLS', 'iFlapy', 'ShadoKida', 'VenomHss010', 'titom9527', 'KmoStaz', 'Xehanort

In [29]:
# Remove this player's name from the list
players_encountered.remove('Jedı07')

['Plus3Hundo', 'Homiechillax', 'Fencepuppet', 'Cermit365', 'blikblak', 'Will BB', 'àntx', 'Diu Da Will BB', 'Domz01', 'SugarHighs', 'Tuna Davis', 'Mattyboyywonder', 'chimuru', '24oz Cold Brew', 'Jessefcd', 'SunxRise', 'YTyukii', 'Nov4yn', 'jetstream jordan', 'Inconvenience', 'Unknown Father', 'Znulodsa', 'Rydur01', 'Mega Adhesive', 'hehexd tyler1', 'chubzhuh', 'ForeverDark2012', 'Issvc', 'Bryanxq3', 'cantivskeitaro', 'Hardsruck', 'hiwi', 'Ozymandias774', 'PostViral', 'The Cosby Suite', 'Zatagon', 'cutiejam', 'PrettyCuteDaddy', 'TheLordEsus', 'zhlhahaha', 'DrDoughnutDude', 'Natebgreat', 'maorT', 'Phangphxn', 'Šampp', 'Jiimoto', 'o Wayfinder o', 'Daddy Skii', 'Skeptical Otter', 'drkeagle98', 'Thaifi', 'Outcome', 'Kraken Mutes ALL', 'Mr Chiwi', 'LCS Miracle Run', 'Harpooneerx', 'Wabby Babbo', 'FrostyCake', 'DooDoo Deluxe', 'ZhouXingwen', 'ShinigamiGhoul', 'BumsAreCool', 'THEGTCGU111', 'Bobgrey', 'SlenderLS', 'iFlapy', 'ShadoKida', 'VenomHss010', 'titom9527', 'KmoStaz', 'Xehanort565', 'Abh

In [31]:
# Create directories for every player and save their match history within
general_dir = '../data/'

for index in range(len(players_encountered)):
    player_dir = general_dir + players_encountered[index] + '_match_history'

    # Create player directory
    if os.path.exists(player_dir):
        shutil.rmtree(player_dir)
    os.mkdir(player_dir)

    # Get the player's match history
    player_history = watcher.match.matchlist_by_puuid(region, match_participants[index], queue=420, count=100)

    # Save match history as json
    player_history_file_path = player_dir + '/' + players_encountered[index] + '_match_history.json'
    player_history_json_string = json.dumps(player_history, indent=2)
    player_history_json_file = open(player_history_file_path, 'w')
    player_history_json_file.write(player_history_json_string)
    player_history_json_file.close()

    if (index%100==0):
        print(index)

0
100
200
300
400
500
600
700


In [5]:
# temporary variable to avoid re-executing
players_encountered = ['Plus3Hundo', 'Homiechillax', 'Fencepuppet', 'Cermit365', 'blikblak', 'Will BB', 'àntx', 'Diu Da Will BB', 'Domz01', 'SugarHighs', 'Tuna Davis', 'Mattyboyywonder', 'chimuru', '24oz Cold Brew', 'Jessefcd', 'SunxRise', 'YTyukii', 'Nov4yn', 'jetstream jordan', 'Inconvenience', 'Unknown Father', 'Znulodsa', 'Rydur01', 'Mega Adhesive', 'hehexd tyler1', 'chubzhuh', 'ForeverDark2012', 'Issvc', 'Bryanxq3', 'cantivskeitaro', 'Hardsruck', 'hiwi', 'Ozymandias774', 'PostViral', 'The Cosby Suite', 'Zatagon', 'cutiejam', 'PrettyCuteDaddy', 'TheLordEsus', 'zhlhahaha', 'DrDoughnutDude', 'Natebgreat', 'maorT', 'Phangphxn', 'Šampp', 'Jiimoto', 'o Wayfinder o', 'Daddy Skii', 'Skeptical Otter', 'drkeagle98', 'Thaifi', 'Outcome', 'Kraken Mutes ALL', 'Mr Chiwi', 'LCS Miracle Run', 'Harpooneerx', 'Wabby Babbo', 'FrostyCake', 'DooDoo Deluxe', 'ZhouXingwen', 'ShinigamiGhoul', 'BumsAreCool', 'THEGTCGU111', 'Bobgrey', 'SlenderLS', 'iFlapy', 'ShadoKida', 'VenomHss010', 'titom9527', 'KmoStaz', 'Xehanort565', 'AbhiOne', 'SenpouRendan', 'Cyrgg', 'SolidOcelot', 'SKT T1 Solo', 'Fr0mHellWithL0ve', 'Dawdles', 'No Parlay', 'J821 Skairipa', 'FM MOOD', 'ShadowedSpirit', 'Mora Azul', 'Yuevboo', '2Scared2Fap ', 'DunqMetro', 'TimothyAlex', 'ShadowedPengu', 'HealthCareHer0', 'Hypokriite', 'El Compi Angel', 'Ivorik', 'Jauruss', 'kayleeen', 'AMERICAN AEGlS', 'STARSCREAIVI', 'K1000', 'We Trippy Mane', 'hEllShAcKLEs', 'BLITZ 1 TRICK 01', 'BRIAN DEad CHAMP', 'elReventón', 'Snowcry7', 'jamo18', 'PRîMĘ', 'LividPeanut', 'twixenstein', 'Swash', 'Liiiisa', 'DrGooglepants', 'Pentameme', 'MangoMercy', 'Qiao Bi Luo ', 'yumy player', 'Pulpy Milk', 'PoliteUsableMutt', 'Mr Obtuse', 'dianayud', 'Ferosit', 'SphinterTickler', 'Bluejrs', 'Castielet', 'Dr Cism', 'Lilriblet', 'riven aficionado', 'Micheal Wave ', 'Artex196', 'Boots n Biscuits', 'GoyMeetsWorld', 'it Ohh', 'Chel0oo', '0h Sh', 'lorddarthroro', 'HMcKringleberry', 'SethiousBlack', 'ShinoDdragon', 'YgVriel', 'SammyTae', 'Jokaeryje', 'BIunt With Trudo', 'SloshyFungGuy', 'RayyyLmao', 'TheWarrior2728', 'GetGoldGet500', 'UrWeakInThePants', 'deader than dirt', 'sprayn', 'Orrelios', 'LaughingFang', 'Randumbweirdo', 'MacTSS2212', 'Labugzorg', 'scrapdizzle', 'Sinister BIade', 'Ooranges', 'Peong', 'Baajabbuss', 'MyAltToLearnShen', 'PentaKuehl', 'Axizon', 'failúre', 'Tunning Crow', 'Lem0nades', 'Tilt2g', 'Headkraken', 'ToxicJuiceBox', 'Pomegranate Milk', 'Awosome1', 'ICreatedLove', 'Kai Jun Seong', '4th Shot Enjoyer', 'ELEGANTKITSUNE', 'uzi way', 'WPGG Habibi', 'XxDarkSasukexXx', 'endermqn125', 'SPD Dindy', 'Zigzagzombie', 'MiamiGgot', 'Lentor', 'CrankDatShiet', 'Lightningbolt420', 'Fishmanlok', 'MeowCatPat', 'silverjnger', 'La Jatte Seurat', 'monet lily pads', 'Daddy Zap', 'Aslaforhade', 'livesean', 'NamFlashback', 'RamadanDeathcap', 'Vex Offender', 'AhWoah', 'SupportCarryOnly', 'wyattrix', 'RottedSocrates', 'HuntToKill', 'Hoidini', 'SnowyHare96', 'Cody Martins', 'orgasmes donor', 'Zawarudo97', 'BigMike66', 'fooooootprint', 'waytogoshadow', 'AsP00n', 'olih', 'sodlfmadmsalsrb', 'WalksWithALimp', 'EstrogenDiff', 'Sir Yehweh', 'MeatyCentipede', 'ThnkyoufrthVenom', 'bobaFLEXED', 'TigerAmbassador', 'Koopz', 'PhongHong', 'Bricklayer622', 'NessTU', 'Burbërry', 'PikachuYou23', 'GrimBlue', 'Kov Shepard', 'GetDonged25', 'estapie', 'xo 991', 'Jama 7', 'AnarchyReign09', 'blackstone2six', 'Kíng Kong', 'MisterDahmer', 'butterbiscuit', 'Prokter', 'Jünë', 'Nicktenine', 'haozhudafeiji', 'PunishedKebab', 'Drak0n3', 'BluluWaffle', '2006batman', 'Dukt', 'veryLIYU', 'HalfTrack024', 'Karêem', 'Jerry West', 'CLYUBARI', 'Battlecast Bear', '69padilla69', 'IamBroder', 'KingLuxe', 'CounterClock92', 'HeckaFrick', 'Asheelarry', 'GETOWNDBITHC', 'EKP l ELFXBLADE', 'PrimarinaRose', 'MIDSUMMERDR3AM', 'Direwolf45', 'ÜnderCøver Gªs', 'Deaf Lee Sin', 'FriedSteakz', 'ProZ Jbox', 'SpankinU', 'Yellow Cloud', 'Taelaren', 'DeathMirage96', 'Fatsio', 'Explocivo', 'TheBabee', 'MylilOznog', 'PolskiOrzelPL', 'AssGoBoing', 'LeoTrickster', 'B0LT Action', 'Zacman', 'Pyp Pyp', 'DJ 3rd nip', 'Fanta Panda', 'ManCakeRobot', 'Lord Butt III', 'delfrona', 'Epicdegree', 'Cdpowder', 'gmdave', 'Impressionnant', 'plywoods', 'HeftyMichael', 'Kimtoness', 'BabyKarats', 'BOOBATHEGREAT', 'HaliPotatoAim', '0impact M1d', 'Maylia', 'FREEWAVE 6', 'Wakaipanda', 'Meetope', 'SpellMishap', 'HedgiesBoBedgies', 'brittlesiren224', 'DunnEasy', 'Otaku Waifu', 'report jg guys', 'StopBeinToxicPlz', 'funnelweb', 'ping me go int', 'scrubmuhballs', 'PolishW0nder', 'redned23', 'Oppel', 'RagingPenguin', 'CaptainCrap', 'AmandaGraci1', 'Joe R Biden Jr', 'switz', 'michaelScotch', 'Snipress', 'RdyMck', 'Armydino', 'amleth', 'Ace Heret0fore', 'METROID4U', 'MjnkKuann', 'DIESOFCRlNGE', 'DemonicBlank', 'poogzzz', 'buenas noches', 'Zeromagamen', 'LazyxxxCuDen', 'Halodjinn', 'R1skTh3Thr0n3', 'Kappasuna', 'paisano gamer', 'Billy Big Balls', 'Dime13', 'Hamoniac', 'Lechuguitaa', 'havokgaming', 'UtterDisgust', 'FLYYOUROWNFLAG', 'banhbao stillfly', 'futher', 'banhbaoboi', 'l3ftyk1ng', 'Latin Refugee', 'Loli Beat', 'Trap God', 'MissAngelic', 'Mind Over Mana', 'iTranqui1ity', 'Leî', 'GiveMeThatGooch', 'StoliCanoli', 'KeePURBreathK', 'phokingaye', 'Grimmace6', 'Cłever', 'Pooossey', 'iamitter', 'the amigo', 'Freshslate', 'FastSlowLoris', 'Gangoon', 'jahh808imi', 'Parallelogrammar', 'Spewcup', 'Karthy', 'Voodoo Soulja', 'Sleépy', 'avitious', 'GXO0JcwWbz', 'Cap Straw Hat', 'Ciyuar', 'Brilliancy Me', 'The Exiled Waifu', 'Fecal Toes', 'Get Rule12b6ed', 'Toxic Graves', 'XOXOJOY', 'Philptheguy', 'mariJANNA waifu', 'NjGoat', 'GIORN0 GI0VANNA', 'ngon vo solo', 'Kaughnye', 'xxWILSONxx', 'Dog motif', 'IlIuminatii', 'BrotherChe', 'lunietunes', 'Spikeypacman', 'ExoviteChinese', 'Lord Juicifer', 'chromosome diff', 'HMC', 'Lagim', 'TotallyNotSmurf4', '1Menace2Society', 'Mr Onion Seller', 'bby georgia', 'jiangshangbaba', 'SparkySpartan', 'Teboose', 'Misery666', 'yvngNub', 'Vinpon', 'Jungle Wukong ', 'boo boo slaya', 'Thresh Records', 'Yuuki Sekai', 'Oghenekevwe', '3 Daydreaming', 'Silver Skywalker', 'jrxjgjcj', 'LOVEX PLOSION 69', 'Livemyga', 'LucidZombie', 'illewal', 'youngzgeg', 'douche de louche', 'i might be toxic', 'i make the rules', 'Word ExE', 'Un Pamplemousse', '     the grabber', 'DadBod Dave', 'Flavordaddy', 'Omar Zi Savage 2', 'TravisŽ', 'ILockgod', 'Jus Ryze', 'BraumsTiddyLover', 'king kunta', 'Foxmix02', 'Vhoxx', 'Elias5613', 'Sinnwin', 'Stormbreaker105', 'Respectoday', 'Booddda', 'sungpi', 'Death2moo', 'Strong Terry', 'TaoLay', 'É O ALCOOL', 'Gigo', 'Jorichen', 'Scoopbb', 'Floppawoppaa', '2JLEE  ', 'Aginør', 'fishbulb07', 'BrayFlex', 'Kaynivore', 'PoundC4ke', 'Choke me Vi', 'Achilles33', 'BETLOG MO', 'AhriUsesCharm', '11114932126v2', 'Anser', 'kdaddy66', 'CHICKENCURRY65', 'CrypticFlare20', 'BeefyChez', 'Frozenier', 'Knicks Reborn', 'Alisong XD', 'ICYpineapple77', 'Cyboru', 'QMS Lao Si Ji', 'StrawHatLuffe', 'Chipper Lad', 'Adrielll', 'FREDDYFAZBEAR87', 'Mside Grizzly', 'thedarkpaladin32', 'subby bf', 'Harry Stylez', 'MythicalGnome', 'yngyioyyt', 'Ostalgie', 'MagicianMatt', 'Reckless318', 'THE BIGGEST DAM', 'Astris Lex', 'DEMACIA STANDS', 'kdragon648', 'Calthiel', 'Nickolette', 'ElectriNerdDonut', 'CaladboIg', 'Jun2893', 'Xerisiel', 'JokesterXO', 'Jintl', 'CUCK0LD', 'SirGamezValorous', 'Sir Alrik Ander', 'True Gringo', 'Maxforce12', 'kDaOKent', 'LucidRaccoon', 'Ranger Crav', 'ChinaNegativeNew', 'Chill I got this', 'RampaginHobo', 'SoFlySenpai', 'CrunkBandicoot17', 'Golden Thunder', 'symbiotePRIME', 'Vichi02', 'YOUr2Heavy4ME', 'Steezypop', 'rippintides', 'IAwwsteen', 'Ba Bánh Bò', 'SoloGuezi', 'Nexx27', 'Krenox', 'TrueVirtuosity', 'Stoned Clefairy', 'Monster Sheng', 'blackmage8bit', 'Havilliard', 'N Luuser', 'l3RENNAN', 'Ne1sonism', 'Hiriko', 'KEVORGOD', 'Trolling Main', 'XDlucaripXD', 'OwHellYeah', 'FZY', 'SierraMikeDelta', 'tyusjdn', 'LukeNukemV', 'D1ORDIOR', 'Anickolas', 'SimpleSim', 'garou5', 'BadLemonNoHope', 'asklv', 'ToastToastToast', 'Kelvinzbb', 'Aromaster', 'PrincessPetch', 'umjimo', 'cccaesar', 'PROJECTNUTSACK1', 'Sir Supportsalot', 'xXxxooXo', 'kFbRRon', 'pnvstbl', 'Mester2242', 'BL22', 'MISSLEXA', 'Script of Dark', 'DoinA', 'Thunderballs7', 'PickledDillz', 'Damac1anStandard', 'NobodyAxel', 'StabbHappyy', 'SLASH EFF EFF', 'DarkStoneMaster1', 'Kcoy', 'InnateOcean', 'BiPower', 'CoG Solarxninja', 'Turtlagon', 'ThirdPlains', 'Canuckerz', 'JOHAL1', 'sit0429ga', 'BanserBro', 'XxMonwamxX', 'CrypticBombs', 'F1R3FOXX21', 'MinionUnionRep', 'Alch Your Ego', 'Polomobo', 'RezonCates', 'AnthonyJustin', 'DrDoritoz', 'rinchuu', 'CozyKayn', 'eneaRoSoNeRo', 'Feetus Swallower', 'Temp346237', '1533', 'Evelyene Sama', 'jumpman255', 'bigdikbandit98', 'Maim The Support', 'Moyaman', 'HughJassNuts', 'Finding Teemò', 'nutbutdonetella', 'Heavens Night', 'Zygahh', 'Annikitty', 'JCTheBeast', 'Hoodrich17', 'Dsminist', 'Bathtubsweat', 'zvbvn', 'qxtkoo', 'Thornfin', 'MLS', 'xMeachx', 'Zalarlenesna', 'SimplyRiley', 'Slightly Lesbian', 'Just a RockK', 'xylulose', 'I Am The Circus', 'Alvisaarthad', 'KiIIaCam', 'OneBOOM', 'TheWaterBoy69', 'sippinchoccymilk', 'Willy Trees', 'Dylan is Dead', 'Tehgonz', 'kk cruisin', 'Cas Bee Farm', 'Vanìsh', 'StewyBuds', 'Moraein', 'TheDravenmeister', 'ScertNmurt', 'oRevert', 'Digital Dagger', 'scooty17', 'Finbar2', 'Rising Wind', 'subvi', 'Cawos', 'Feedonomics', 'Khaegar', 'MV Shcneller ', 'Kremist', 'xxwwigxwwagxx', 'KoukiThePeasant', 'cuuiyygc', 'shadowdwarf III', 'Timmychunga', 'Senju2', 'faketailopez', '  Crazzy Puppy  ', 'GrosBumSale', 'terminationator', 'tonyjack', 'YvesSaintLaurent', 'Siion', 'koolpis', 'L3Knight', 'Quínn', '1m batman', 'sir zarcon', 'corn99', 'Of The Valar', 'LouDowg', 'Coochie Desire', 'xInkredible', 'waterdrinker12', 'brenning15', 'Zarathuxtra', 'briochebun', 'Saint Inca', 'GANGROCK', 'prettylites', 'Dindynn ', 'Wolfiewood', 'OMGaFnBear', 'Krypt00', 'Sysliana', 'Drazha', 'a nasty mess', 'Bloody Sanguine', 'MrDrBrownThumb', 'Cragonator', 'LordJustice95', 'TheUndisputed', 'TheeCatzMeow', 'OverZelus', 'RiptidaL', 'PARK SEROI', 'victor', 'Onefasterboy', 'quagberly', 'wedbaron', 'Satiety', 'Dopest', 'SADDAMHUSS1NGED', 'silentdeath143', 'Tye Rex', 'Moonmac', 'KeithShaley', 'Milkywater3222', 'Kervnin', 'JCDrizzy', 'IWINYOUL0SE', 'Ca1cu1atedError', 'Saidku', '08Beelzebub', 'IamYuzo', 'Aram Goat', 'kong1', 'Savior Self', 'shrunknmonky', 'Refreshments', 'FireBelowTheMtn', 'TribalGawd', 'Ecktoplasm', 'LordBeMyShield', 'KSS Avenge', 'KSS Bunny', 'Mystogo', 'brodekeiser', 'OffTiltGhoul', 'Drintas', 'Shadeslayer998', 'Shadowthemagez', 'râw', 'FYI x iSlay', 'pizzzatoes', 'Cereal4Dinner', 'C9 SchruteFarms', 'xardespl', 'KentheADC', 'malevolentshrin3', 'Tanchy', 'SpaceLion14', 'Intetsrasu', 'YvaGr33d', 'Infinite Void', 'LewdKitsune', 'Perseverant', 'Winstang', 'Kazan', 'Raze01', 'ZackTattack', 'Garbage Riven', 'MD0T', 'SwanZam', 'Aaroń', 'Exijun', 'DEADAIM13', 'FutureBobcat25', 'Scivir', 'Fâst', 'ARISE QUEEN', 'SeaKoala', 'Fahzi', 'yowoodzm', 'Washed Glizzy', 'FishuFreshu', 'BodMang Zeru', 'Love Dumper', 'Bussydefiler', 'Fall3n Soldi3r', 'Blxckgate', 'Crossingguard', 'OSav', 'Dumptime', 'HannibalBarca12', 'CholyArt', 'DrippingDroopies', 'Alegsus', 'Ray Yawn', 'FernandoOhtani', 'LightningGod1738', 'XxGrakynxX', 'VivoVitro', 'RandelI', 'ryeung1', 'KevinChangSz', 'DaddyyDraven', 'LaCuboidVoid', 'alex1001458', 'Frost Prism', 'Worst Silver NA', 'Robotic M1dget', 'BattleXD', 'RRnaiNwD', 'Recline ', 'Ambushee', 'Cocostanley', 'Kehaiji', 'dreamweaver86', 'aizaiff', 'Perculez', 'FringoMcDingo']

general_dir = '../data/'

# Create directories for each player's games and pull the games' data to those directories
for index in range(len(players_encountered)):
   # Directory of the player's match history
   history_dir = general_dir + players_encountered[index] + '_match_history/' + players_encountered[index] + '_match_history.json'
   matches_dir = general_dir + players_encountered[index] + '_match_history/matches' 

   # Create matches directory if it doesn't already exist
   if not (os.path.exists(matches_dir)):
      os.mkdir(matches_dir)

   # Open player's match history file, and cycle through the list's values to create a directory per game
   with open(history_dir) as history_file:
      matches = json.load(history_file)

      for match in matches:
         # Directory where the match data will be stored
         game_dir = general_dir + players_encountered[index] + '_match_history/' + 'matches/match_' + match
         match_file_path = game_dir + '/' + match + '.json'

         # Only make the pull request if the match data file does not already exist
         if not (os.path.exists(match_file_path)):
            # Create match directory
            if os.path.exists(game_dir):
               shutil.rmtree(game_dir)
            os.mkdir(game_dir)

            # Try block is necessary in case the match is a "ghost game". View Documentation for more information
            try:
               match_data = watcher.match.by_id(region, match)

               # Save match data
               match_data_json_string = json.dumps(match_data, indent=2)
               match_data_json_file = open(match_file_path, 'w')
               match_data_json_file.write(match_data_json_string)
               match_data_json_file.close()
            except:
               os.rmdir(game_dir)
               alert = match + ' is broken. Skipping.'
               print(alert)
   
   alert = 'Player #' + str(index) + ', ' + players_encountered[index] + ' completed.'
   if (index%50==0):
      print(alert)

Player #0, Plus3Hundo completed.
Player #50, Thaifi completed.
Player #100, BRIAN DEad CHAMP completed.
Player #150, MacTSS2212 completed.
NA1_4209936171 is broken. Skipping.
Player #200, Cody Martins completed.
Player #250, KingLuxe completed.
Player #300, DunnEasy completed.
Player #350, Leî completed.
Player #400, jiangshangbaba completed.
Player #450, Aginør completed.
Player #500, Maxforce12 completed.
Player #550, Sir Supportsalot completed.
Player #600, Zygahh completed.
Player #650,   Crazzy Puppy   completed.
NA1_4035484363 is broken. Skipping.
Player #700, Milkywater3222 completed.
Player #750, FutureBobcat25 completed.


## Open the cleaned ML-friendly match data file as a Dataframe

In [174]:
df1 = pd.read_csv('../data/Jedı07_match_history/matches/match_NA1_4034399637/ml_data.csv',delimiter=',')
df2 = pd.read_csv('../data/Jedı07_match_history/matches/match_NA1_4034434056/ml_data.csv',delimiter=',')
champion_data = None

with open('../champion_data/champion.json','r',encoding='utf-8') as f :
    champions = json.load(f)


df = pd.DataFrame()

def add(old,new_df) :
    appended_df = pd.concat([old,new_df],ignore_index=True)
    return appended_df

def champ_name_replacer(champ_name) :
    champ_data = champions['data']
    return champ_data[champ_name]['tags'][0]

df = add(df,df1)
df = add(df,df2)

print('BEFORE TAG REPLACEMENT: \n',df[['dmg_carry_0','dmg_carry_1']],'\n')

columns = list(df.columns)
champ_name_columns = [c for c in columns if c.startswith('team_comp') or c.startswith('dmg_carry') or c.startswith('obj_carry')]


for c in champ_name_columns :
    df[c] = df[c].apply(lambda x : champ_name_replacer(x))


print('AFTER TAG REPLACEMENT: \n',df[['dmg_carry_0','dmg_carry_1']],'\n')


unique_champ_tags = df[champ_name_columns].stack().unique()
dummy_dict = {}
for k in champ_name_columns :
    dummy_dict[k] = unique_champ_tags.copy()
dummy_df = pd.DataFrame(dummy_dict)
extra_rows = dummy_df.shape[0]

df = add(df,dummy_df)
print(df.loc[:1,champ_name_columns].values)

x = df[champ_name_columns]
ohe = OneHotEncoder()
x = ohe.fit_transform(x).toarray()


total_size = len(x)
x = x[:total_size-extra_rows]

print(x)





BEFORE TAG REPLACEMENT: 
   dmg_carry_0 dmg_carry_1
0       Vayne      Akshan
1        Ryze       Nasus 

AFTER TAG REPLACEMENT: 
   dmg_carry_0 dmg_carry_1
0    Marksman    Marksman
1        Mage     Fighter 

[['Marksman' 'Fighter' 'Fighter' 'Fighter' 'Mage' 'Marksman' 'Mage'
  'Marksman' 'Fighter' 'Mage' 'Fighter' 'Marksman' 'Marksman' 'Support']
 ['Mage' 'Fighter' 'Support' 'Fighter' 'Mage' 'Marksman' 'Mage' 'Fighter'
  'Marksman' 'Fighter' 'Fighter' 'Assassin' 'Marksman' 'Tank']]
[[0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0

In [175]:
### call labelencoder.fit and transform for x_train unique champs and call labelencoder.transform for x_test to avoid data leakage